In [ ]:
import plotly.graph_objects as go
import plotly.offline as py
import plotly.express as px
from plotly.subplots import make_subplots

import numpy as np
from numpy import argmax
from praatio import textgrid
import matplotlib.pyplot as plt
import soundfile as sf
from gudhi.point_cloud import timedelay
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from ripser import ripser
from persim import plot_diagrams
import plotly.io as pio
import time
%matplotlib qt5

# head_tail_scissor is to erase signal in head and tail that has amplitude smaller than 0.05
# can also use it to see if the length of renewing signal is greater than 500 or not 
def head_tail_scissor(sig):
    valid_interval=[index for index in range(len(sig)) if (sig[index]>0.03)]
    if len(valid_interval)==0:
        return False,sig
    head=min(valid_interval)
    tail=max(valid_interval)
    sig=sig[head:tail+1]
    if tail-head<500:
        return False,sig
    return True,sig

# principle_frequency_finder is to find the period of a speech signal
def principle_frequency_finder(sig):
    t=int(len(sig)/2)
    corr=np.zeros(t)

    for index in np.arange(t):
        ACF_delay=sig[index:]
        L=(t-index)/2
        m = np.sum(sig[int(t-L):int(t+L+1)]**2) + np.sum(ACF_delay[int(t-L):int(t+L+1)]**2)
        r = np.sum(sig[int(t-L):int(t+L+1)]*ACF_delay[int(t-L):int(t+L+1)])
        corr[index] = 2*r/m

    zc = np.zeros(corr.size-1)
    zc[(corr[0:-1] < 0)*(corr[1::] > 0)] = 1
    zc[(corr[0:-1] > 0)*(corr[1::] < 0)] = -1

    admiss = np.zeros(corr.size)
    admiss[0:-1] = zc
    for i in range(1, corr.size):
        if admiss[i] == 0:
            admiss[i] = admiss[i-1]

    maxes = np.zeros(corr.size)
    maxes[1:-1] = (np.sign(corr[1:-1] - corr[0:-2])==1)*(np.sign(corr[1:-1] - corr[2::])==1)
    maxidx = np.arange(corr.size)
    maxidx = maxidx[maxes == 1]
    max_index = 0
    if len(corr[maxidx]) > 0:
        max_index = maxidx[np.argmax(corr[maxidx])]

    return (max_index, corr)

In [ ]:
inputFile="E:\\Datasets\\speech_file\\article_TDA_phonetic_output\\ALL_049_F_ENG_ENG_HT1.Textgrid"
tg=textgrid.openTextgrid(inputFile,includeEmptyIntervals=False)
phoneTier=tg.getTier('phones')

specified_vowel=['m']
wavFile='E:\\Datasets\\speech_file\\article_TDA_phonetic_input\\ALL_049_F_ENG_ENG_HT1.wav'
M=100
print('dimension = ',M)
max_edge_length=1

sig,samplerate=sf.read(wavFile)
specified_vowel_list=[ele for ele in phoneTier.entries if ele[2] in specified_vowel]

# wav_fraction_finder is to find the corresponding wav signal according to interval
def wav_fraction_finder(start_time, end_time):
    sig_fraction=sig[int(start_time*samplerate):int(end_time*samplerate)]
    return sig_fraction

specified_valid_vowel_list=[head_tail_scissor(wav_fraction_finder(ele[0],ele[1]))[1] for ele in specified_vowel_list if head_tail_scissor(wav_fraction_finder(ele[0],ele[1]))[0]]
print('There are ',str(len(specified_valid_vowel_list)),' phones in the specified list.')

T=[0]*len(specified_valid_vowel_list)
for i in range(len(specified_valid_vowel_list)):
    T[i],corr=principle_frequency_finder(np.array(specified_valid_vowel_list[i]))
    T[i]=T[i]

delay=[round(ele*6/M) for ele in T]
for element in range(len(delay)):
    if delay[element]==0:
        delay[element]=1


In [ ]:
element=5
skip=1
print('delay = ',delay[element])
data=specified_valid_vowel_list[element]
if delay[element]*M>len(data):
    delay[element]=int(np.floor(len(data)/M))
if delay[element]==0:
    delay[element]=1
point_Cloud=timedelay.TimeDelayEmbedding(M, delay[element], skip)
Points=point_Cloud(data)
X=StandardScaler().fit_transform(Points)
pca=PCA(n_components=3,whiten=True)
X_PCA=pca.fit_transform(X)
print(pca.explained_variance_ratio_)


point_Cloud=timedelay.TimeDelayEmbedding(M,delay[element],skip)
Points=point_Cloud(data)
print(len(Points))
t=time.time()
dgms = ripser(Points,maxdim=1)['dgms']
print('The computation time = ',time.time()-t)
plot_diagrams(dgms,lifetime=True)

dgms=dgms[1]
persistent_time=[ele[1]-ele[0] for ele in dgms]            
index=argmax(persistent_time)
birth_date=[ele[0] for ele in dgms]
lifetime=persistent_time[index]
print('maximal lifetime = ', lifetime)

fig = go.Figure(data=go.Scatter3d(
    x=X_PCA[:,0], y=X_PCA[:,1], z=X_PCA[:,2],
    marker=dict(
        size=4,
        color=X_PCA[:,2],
        colorscale='Viridis',#'sunset','thermal',
    ),
    line=dict(
        color='#0000b3',
        width=1
    )
))

fig.update_layout(
    width=800,
    height=800,
    template='plotly',
    autosize=False,
    scene=dict(
        camera=dict(
            up=dict(
                x=0,
                y=0,
                z=1
            ),
            eye=dict(
                x=1.2,
                y=1.2,
                z=1.2,
            )
        ),
        aspectratio = dict( x=1, y=1, z=0.7 ),
        aspectmode = 'manual',
    ),
)
fig.show()
#pio.write_image(fig, 'pdf_graph_raw/PCA_S'+str(skip)+'.pdf',scale=6, width=1080, height=1080)
#py.plot(fig, filename='vector_graph/M_'+str(M), image='svg')

In [ ]:
ticket = ['skip=1', 'skip=5','skip=10']
computation_time=[0.406872034072876,0.004989147186279297,0.001003265380859375]

fig = make_subplots(
    rows=1, cols=3,
    column_widths=[0.3,0.3,0.3],
    row_heights=[1],
    specs=[[{"type": "bar"}, {"type": "bar"},{"type": "bar"}]])

# Add scattergeo globe map of volcano locations
fig.add_trace(go.Bar(
    x=ticket,
    y=computation_time,
    name='Computation Time (seconds)',
    text=[0.407,0.005,0.001],
    textposition='auto',
    marker_color='indianred'),
    row=1, col=1
)

# Add locations bar chart
fig.add_trace(
    go.Bar(x=ticket,
    y=[0.8699688576161861,0.8189932927489281,0.7689457535743713],
    name='Maximal Persistence',
    text=[0.87,0.82,0.77],
    textposition='auto',
    marker_color='lightsalmon'),
    row=1, col=2
)

# Add 3d surface of volcano
fig.add_trace(go.Bar(
    x=ticket,
    y=[491,99,50],
    name='Number of Points',
    text=[491,99,50],
    textposition='auto',
    marker_color='#ffb84d'),
    row=1, col=3
)

fig.update_layout(
    width=800,
    height=200,
    margin=dict(r=1, t=1, b=1, l=1),
    template='simple_white',
)

fig.show()

pio.write_image(fig, 'skip_bar_raw.pdf',scale=1, width=1080, height=300)
